In [1]:
import numpy as np
import yfinance as yf
import pandas as pd
from math import sqrt
import tasas_vasicek as tv

In [2]:
empresas=pd.read_excel('data_final.xlsx')

In [3]:
symbols=empresas['symbol']

In [4]:
class portfolio:
    def __init__(self,symbols):
        self.symbs=sorted(symbols)
    def get_info_compamys(self):
        data=yf.download(tickers=self.symbs, start='2019-6-1',progress=0)
        data=data.dropna(axis=0)
        data=data[['Open', 'Close', 'High', 'Low']]
        return data
    @property
    def Close(self):
        return self.get_info_compamys()['Close']
    def matriz_rendimientos(self):
        def rendimiento(comp): return np.diff(np.log(comp))*100 # return [log(comp[i]/comp[i-1]) for i in range(1,len(comp))]
        df_rend=self.Close.apply(rendimiento,axis=0)
        df_rend.columns.name='Empresa'
        return df_rend
    def rend_volt_anual(self):
        rendimientos=self.matriz_rendimientos()
        medias=pd.DataFrame(rendimientos.mean()*365)
        varianzas=pd.DataFrame(rendimientos.std()*sqrt(len(rendimientos)*360/(len(rendimientos)+1))) 
        df=pd.merge(medias,varianzas,on='Empresa')
        df.columns=["Media","Varianza"]
        return df
    def cuartiles(self):
        df_cuart=self.matriz_rendimientos().describe()
        df_cuart.index.name="Cuartil"
        return df_cuart.iloc[-4:-1]
    

In [5]:
my_portfolio=portfolio(symbols[:12])

In [6]:
my_portfolio.matriz_rendimientos()

Empresa,AAPL.MX,AMZN.MX,BRKB.MX,GOOG.MX,JNJ.MX,JPM.MX,META.MX,MSFT.MX,TSLA.MX,TSMN.MX,UNH.MX,V.MX
0,2.538614,1.429540,0.688164,0.573650,0.271164,2.130258,1.490629,0.915340,6.577713,0.000000,-0.893011,1.423384
1,1.768198,0.611720,1.086281,-1.185662,0.648357,0.934730,0.292563,2.693541,2.618733,0.000000,1.669120,2.181319
2,1.555915,1.332824,1.000008,0.039284,2.022171,0.506820,0.402876,1.906539,4.753393,0.000000,0.000000,1.139129
3,2.531440,2.671104,0.585351,2.635551,0.980672,-1.488400,2.897762,2.943660,-1.386709,6.039522,2.162193,2.035066
4,-0.602091,1.292516,-1.459810,-0.450468,-1.753076,-0.187617,-0.975329,-1.371371,2.405786,0.536121,-1.294804,-1.320730
...,...,...,...,...,...,...,...,...,...,...,...,...
841,-2.179437,-1.429970,-0.871638,-1.410516,0.419869,-0.713468,-0.440995,-2.196888,-1.368969,-0.537539,0.000000,-1.349559
842,1.594547,1.469661,1.363780,2.356627,-1.973836,1.974010,1.280048,2.352272,-9.195819,-0.107855,1.210348,2.390950
843,1.857326,3.903156,2.365264,2.155416,0.567162,3.713538,0.812398,2.535233,1.669633,4.432664,0.221893,1.111305
844,0.897658,0.999377,0.000000,0.826961,0.000000,-1.276651,0.054356,1.090208,-2.836229,2.706335,1.575512,1.843517


In [7]:
my_portfolio.rend_volt_anual()

,Media,Varianza
Empresa,,
AAPL.MX,53.045267,36.169166
AMZN.MX,16.173839,41.185925
BRKB.MX,15.241032,27.674593
GOOG.MX,30.092521,35.741684
JNJ.MX,9.643046,27.954336
JPM.MX,1.818995,37.411249
META.MX,-6.129587,49.313639
MSFT.MX,31.959706,33.594504
TSLA.MX,130.095764,79.356817


# Matrices de volatidades, varianzas y covarianzas

In [8]:
varianzas=list(my_portfolio.rend_volt_anual()['Varianza'])

In [9]:
Matriz_Si=[[0]*len(varianzas) for i in range(len(varianzas))]
for i,s_i in zip(range(len(Matriz_Si)),range(len(varianzas))):
                Matriz_Si[i][s_i]=varianzas[s_i]
Matriz_Si=pd.DataFrame(Matriz_Si)
Matriz_Si

,0,1,2,3,4,5,6,7,8,9,10,11
0,36.169166,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,41.185925,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,27.674593,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,35.741684,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,27.954336,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,37.411249,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,49.313639,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,33.594504,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,79.356817,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,38.820194,0.000000,0.000000


In [10]:
Matrizcorr=my_portfolio.matriz_rendimientos().corr()
Matrizcorr

Empresa,AAPL.MX,AMZN.MX,BRKB.MX,GOOG.MX,JNJ.MX,JPM.MX,META.MX,MSFT.MX,TSLA.MX,TSMN.MX,UNH.MX,V.MX
Empresa,,,,,,,,,,,,
AAPL.MX,1.000000,0.591134,0.323054,0.514086,0.270438,0.282971,0.534175,0.696350,0.420910,0.349129,0.310869,0.437241
AMZN.MX,0.591134,1.000000,0.232285,0.516701,0.168129,0.178156,0.561314,0.626133,0.376670,0.305994,0.181192,0.304919
BRKB.MX,0.323054,0.232285,1.000000,0.313908,0.381328,0.500350,0.231528,0.329849,0.162459,0.194169,0.346819,0.425433
GOOG.MX,0.514086,0.516701,0.313908,1.000000,0.224021,0.276686,0.543170,0.589883,0.293599,0.302868,0.300338,0.418913
JNJ.MX,0.270438,0.168129,0.381328,0.224021,1.000000,0.156508,0.110717,0.285320,-0.023025,0.071183,0.372273,0.244426
JPM.MX,0.282971,0.178156,0.500350,0.276686,0.156508,1.000000,0.229816,0.298438,0.229427,0.193637,0.296423,0.459376
META.MX,0.534175,0.561314,0.231528,0.543170,0.110717,0.229816,1.000000,0.576722,0.313959,0.247048,0.190822,0.372047
MSFT.MX,0.696350,0.626133,0.329849,0.589883,0.285320,0.298438,0.576722,1.000000,0.412506,0.293296,0.330581,0.507316
TSLA.MX,0.420910,0.376670,0.162459,0.293599,-0.023025,0.229427,0.313959,0.412506,1.000000,0.272960,0.139139,0.239839


In [11]:
Matrizcorr.columns.name=""
Matrizcorr.index.name=""
Matrizcorr.columns=[n for n in range(12)]
Matrizcorr.index=[n for n in range(12)]

In [12]:
MVC=Matriz_Si@Matrizcorr@Matriz_Si.T;MVC


,0,1,2,3,4,5,6,7,8,9,10,11
0,1308.208596,880.588520,323.365886,664.583534,273.435316,382.897863,952.772017,846.124904,1208.124358,490.210270,390.605091,521.393480
1,880.588520,1696.280446,264.759089,760.611996,193.571548,274.505034,1140.043851,866.331272,1231.102192,489.237422,259.244307,414.036434
2,323.365886,264.759089,765.883116,310.498053,295.004821,518.033346,315.973907,306.665382,356.787732,208.602016,333.431468,388.167032
3,664.583534,760.611996,310.498053,1277.467991,223.826835,369.967756,957.365915,708.286800,832.749347,420.229644,372.912320,493.633696
4,273.435316,193.571548,295.004821,223.826835,781.444912,163.677562,152.626441,267.947658,-51.078352,77.247354,361.520303,225.269399
5,382.897863,274.505034,518.033346,369.967756,163.677562,1399.601567,423.983357,375.080126,681.131052,281.220824,385.244697,566.599579
6,952.772017,1140.043851,315.973907,957.365915,152.626441,423.983357,2431.834962,955.436006,1228.638931,472.939335,326.901865,604.881298
7,846.124904,866.331272,306.665382,708.286800,267.947658,375.080126,955.436006,1128.590701,1099.721035,382.500102,385.805159,561.891965
8,1208.124358,1231.102192,356.787732,832.749347,-51.078352,681.131052,1228.638931,1099.721035,6297.504399,840.891885,383.579213,627.495039
9,490.210270,489.237422,208.602016,420.229644,77.247354,281.220824,472.939335,382.500102,840.891885,1507.007456,204.497866,268.194981


In [15]:
def sigmaW(Porc_w): return sqrt(Porc_w.T@np.array(MVC[:-1])@Porc_w)


In [16]:
MVC

,0,1,2,3,4,5,6,7,8,9,10,11
0,1308.208596,880.588520,323.365886,664.583534,273.435316,382.897863,952.772017,846.124904,1208.124358,490.210270,390.605091,521.393480
1,880.588520,1696.280446,264.759089,760.611996,193.571548,274.505034,1140.043851,866.331272,1231.102192,489.237422,259.244307,414.036434
2,323.365886,264.759089,765.883116,310.498053,295.004821,518.033346,315.973907,306.665382,356.787732,208.602016,333.431468,388.167032
3,664.583534,760.611996,310.498053,1277.467991,223.826835,369.967756,957.365915,708.286800,832.749347,420.229644,372.912320,493.633696
4,273.435316,193.571548,295.004821,223.826835,781.444912,163.677562,152.626441,267.947658,-51.078352,77.247354,361.520303,225.269399
5,382.897863,274.505034,518.033346,369.967756,163.677562,1399.601567,423.983357,375.080126,681.131052,281.220824,385.244697,566.599579
6,952.772017,1140.043851,315.973907,957.365915,152.626441,423.983357,2431.834962,955.436006,1228.638931,472.939335,326.901865,604.881298
7,846.124904,866.331272,306.665382,708.286800,267.947658,375.080126,955.436006,1128.590701,1099.721035,382.500102,385.805159,561.891965
8,1208.124358,1231.102192,356.787732,832.749347,-51.078352,681.131052,1228.638931,1099.721035,6297.504399,840.891885,383.579213,627.495039
9,490.210270,489.237422,208.602016,420.229644,77.247354,281.220824,472.939335,382.500102,840.891885,1507.007456,204.497866,268.194981


In [17]:
n=len(MVC)
MVCA=MVC
MVCA.loc[n]=[1]*n
MVCA=MVCA.T
MVCA.loc[n]=[1]*n+[0]
MVCA=MVCA.T
MVCA_inv=np.linalg.inv(MVCA)

In [18]:
MVCA

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,1308.208596,880.588520,323.365886,664.583534,273.435316,382.897863,952.772017,846.124904,1208.124358,490.210270,390.605091,521.393480,1.0
1,880.588520,1696.280446,264.759089,760.611996,193.571548,274.505034,1140.043851,866.331272,1231.102192,489.237422,259.244307,414.036434,1.0
2,323.365886,264.759089,765.883116,310.498053,295.004821,518.033346,315.973907,306.665382,356.787732,208.602016,333.431468,388.167032,1.0
3,664.583534,760.611996,310.498053,1277.467991,223.826835,369.967756,957.365915,708.286800,832.749347,420.229644,372.912320,493.633696,1.0
4,273.435316,193.571548,295.004821,223.826835,781.444912,163.677562,152.626441,267.947658,-51.078352,77.247354,361.520303,225.269399,1.0
5,382.897863,274.505034,518.033346,369.967756,163.677562,1399.601567,423.983357,375.080126,681.131052,281.220824,385.244697,566.599579,1.0
6,952.772017,1140.043851,315.973907,957.365915,152.626441,423.983357,2431.834962,955.436006,1228.638931,472.939335,326.901865,604.881298,1.0
7,846.124904,866.331272,306.665382,708.286800,267.947658,375.080126,955.436006,1128.590701,1099.721035,382.500102,385.805159,561.891965,1.0
8,1208.124358,1231.102192,356.787732,832.749347,-51.078352,681.131052,1228.638931,1099.721035,6297.504399,840.891885,383.579213,627.495039,1.0
9,490.210270,489.237422,208.602016,420.229644,77.247354,281.220824,472.939335,382.500102,840.891885,1507.007456,204.497866,268.194981,1.0


In [20]:
pd.DataFrame(MVCA_inv)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.001798,-0.000271,-4.045919e-05,-0.000048,-0.000163,-5.200653e-06,-0.000147,-6.898484e-04,-9.592437e-05,-0.000150,-0.000079,-0.000108,-0.021019
1,-0.000271,0.001184,-7.103463e-05,-0.000189,-0.000067,5.167046e-05,-0.000214,-4.264876e-04,-4.941131e-05,-0.000102,0.000062,0.000092,0.061068
2,-0.000040,-0.000071,2.028912e-03,-0.000108,-0.000666,-5.742098e-04,0.000001,2.540473e-05,-7.240593e-07,-0.000134,-0.000170,-0.000291,0.195136
3,-0.000048,-0.000189,-1.079843e-04,0.001436,-0.000086,-3.341023e-05,-0.000242,-3.455551e-04,3.628140e-06,-0.000126,-0.000119,-0.000142,0.047494
4,-0.000163,-0.000067,-6.658785e-04,-0.000086,0.001502,6.657890e-05,0.000111,-2.439094e-04,1.130094e-04,-0.000072,-0.000399,-0.000095,0.314220
5,-0.000005,0.000052,-5.742098e-04,-0.000033,0.000067,1.085257e-03,-0.000017,4.466683e-07,-4.566029e-05,-0.000067,-0.000121,-0.000340,0.064290
6,-0.000147,-0.000214,1.492968e-06,-0.000242,0.000111,-1.744562e-05,0.000757,-2.053119e-04,-1.847222e-06,0.000009,0.000026,-0.000076,-0.001858
7,-0.000690,-0.000426,2.540473e-05,-0.000346,-0.000244,4.466683e-07,-0.000205,2.473716e-03,-9.420953e-05,0.000028,-0.000113,-0.000409,0.026144
8,-0.000096,-0.000049,-7.240593e-07,0.000004,0.000113,-4.566029e-05,-0.000002,-9.420953e-05,2.154309e-04,-0.000044,-0.000007,0.000007,-0.004621
9,-0.000150,-0.000102,-1.343365e-04,-0.000126,-0.000072,-6.710563e-05,0.000009,2.808562e-05,-4.381479e-05,0.000741,-0.000049,-0.000034,0.152661


In [27]:
Eri=list(my_portfolio.rend_volt_anual()['Media'])

In [29]:
N=len(MVCA)
MVCAA=MVCA
MVCAA.loc[N]=Eri+[0]
MVCAA=MVCAA.T
MVCAA.loc[N]=Eri+[0,0]
MVCAA=MVCAA.T
MVCAA

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1308.208596,880.588520,323.365886,664.583534,273.435316,382.897863,952.772017,846.124904,1208.124358,490.210270,390.605091,521.393480,1.0,53.045267
1,880.588520,1696.280446,264.759089,760.611996,193.571548,274.505034,1140.043851,866.331272,1231.102192,489.237422,259.244307,414.036434,1.0,16.173839
2,323.365886,264.759089,765.883116,310.498053,295.004821,518.033346,315.973907,306.665382,356.787732,208.602016,333.431468,388.167032,1.0,15.241032
3,664.583534,760.611996,310.498053,1277.467991,223.826835,369.967756,957.365915,708.286800,832.749347,420.229644,372.912320,493.633696,1.0,30.092521
4,273.435316,193.571548,295.004821,223.826835,781.444912,163.677562,152.626441,267.947658,-51.078352,77.247354,361.520303,225.269399,1.0,9.643046
5,382.897863,274.505034,518.033346,369.967756,163.677562,1399.601567,423.983357,375.080126,681.131052,281.220824,385.244697,566.599579,1.0,1.818995
6,952.772017,1140.043851,315.973907,957.365915,152.626441,423.983357,2431.834962,955.436006,1228.638931,472.939335,326.901865,604.881298,1.0,-6.129587
7,846.124904,866.331272,306.665382,708.286800,267.947658,375.080126,955.436006,1128.590701,1099.721035,382.500102,385.805159,561.891965,1.0,31.959706
8,1208.124358,1231.102192,356.787732,832.749347,-51.078352,681.131052,1228.638931,1099.721035,6297.504399,840.891885,383.579213,627.495039,1.0,130.095764
9,490.210270,489.237422,208.602016,420.229644,77.247354,281.220824,472.939335,382.500102,840.891885,1507.007456,204.497866,268.194981,1.0,31.728967


In [32]:
def sigmaW(Porc_w): return sqrt(Porc_w.T@np.array(MVC[:-1])@Porc_w)
Vector_B=np.array([0]*n+[1]).reshape(n+1,1)
W_min_riesgo=MVCA_inv@Vector_B

# Cartera de mínimo riesgo

In [39]:
cmr=pd.DataFrame(W_min_riesgo[:-1])
cmr.columns=["Ponderaciones"]
cmr.index=symbols[:12]
cmr.index.name='Compañia'
cmr

,Ponderaciones
Compañia,
AAPL.MX,-0.021019
MSFT.MX,0.061068
AMZN.MX,0.195136
GOOG.MX,0.047494
TSLA.MX,0.314220
BRKB.MX,0.064290
UNH.MX,-0.001858
JPM.MX,0.026144
JNJ.MX,-0.004621


# Tasa vasicek

In [40]:
tasas_28=pd.read_excel('historico_tasas_cetes.xlsx',sheet_name='28-dias')
tasas_90=pd.read_excel('historico_tasas_cetes.xlsx',sheet_name='91-dias')
tasas_182=pd.read_excel('historico_tasas_cetes.xlsx',sheet_name='182-dias')
tasas_364=pd.read_excel('historico_tasas_cetes.xlsx',sheet_name='364-dias')


In [41]:
rf=tv.tasa(tasas_28['tasa']/100)
rf

0.09835813561518843

In [49]:
Rend_Exces=np.array(Eri)-rf
V_vector=np.linalg.inv(MVC[:-1])@Rend_Exces
W_tan=V_vector/V_vector.sum()

In [53]:
W_tan

array([ 1.11625186, -0.48163483,  0.32532029,  0.46740619, -0.15069347,
       -0.47096691, -0.71201283,  0.17662135,  0.46286731,  0.16337404,
        0.51778245, -0.41431545])